<a href="https://colab.research.google.com/github/Akshitasama/akshita_INFO5731_Spring2021/blob/main/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [1]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium




Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

**Scrapping Data and Converting to Dataframe**

In [3]:
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver=webdriver.Chrome('chromedriver',options=options)
page='https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv'
title=[]
review=[]
driver.get(page)
for x in range(7):
  driver.find_element_by_class_name("ipl-load-more__button").click()
  time.sleep(6)
  title_list= driver.find_elements(By.CLASS_NAME, "title")
  review_list = driver.find_elements(By.CLASS_NAME, "text")
for a, b in zip(title_list, review_list):
      title.append((a.text).replace('\n',''))
      review.append(b.text)
data_frame = pd.DataFrame(list(zip(title, review)), columns =['Title heading', 'Review'])
print("Length of data frame is {0}".format(len(data_frame)))
data_frame

Length of data frame is 200


,Title heading,Review
0,Outstanding movie with a haunting performance ...,"Every once in a while a movie comes, that trul..."
1,Only certain people can relate,This is a movie that only those who have felt ...
2,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o..."
3,MASTERPIECE 😍,Joaquin Phoenix gives a tour de force performa...
4,The Hype is real,Most of the time movies are anticipated like t...
...,...,...
195,Waste of money and time,This movie won't be felt only by those who suf...
196,Self indulgent,No surprises here. This was just the same idea...
197,If I could give -1star....,Disturbing with very little entertainment valu...
198,Worst movie ever,Just went and watched the joker movie and it h...


In [4]:
import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

**Data Cleaning**

**Conveting to Lower Case**

In [5]:
import re
from nltk.corpus import stopwords
from textblob import TextBlob
data_frame['Lower Case'] = data_frame['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data_frame['Removal of Punctuation'] = data_frame['Lower Case'].str.replace('[^\w\s]','')
data_frame['Removal of Special Characters'] = data_frame['Removal of Punctuation'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
s= stopwords.words('english')
data_frame['Stopwords Removal'] = data_frame['Removal of Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in s))
data_frame['Spelling Correction'] = data_frame['Stopwords Removal'].apply(lambda x: str(TextBlob(x).correct()))


**Tokenization**

In [6]:
import nltk
nltk.download('punkt')
data_frame['Tokenization'] = data_frame['Spelling Correction'].apply(lambda x: TextBlob(x).words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Stemming**

In [7]:
from nltk.stem import PorterStemmer
from textblob import Word
import nltk
p_stem = PorterStemmer()
data_frame['After Stemming'] = data_frame['Tokenization'].apply(lambda x: " ".join([p_stem.stem(word) for word in x]))
nltk.download('wordnet')
data_frame['After Lemmatization'] = data_frame['After Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data_frame.to_csv('/content/sample_data/reviews.csv',index=False)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**1.1 Trigrams**

In [8]:
import itertools
import collections
from nltk import ngrams
from nltk.tokenize import word_tokenize

sentence = []
for row in data_frame['After Lemmatization']:
  sentence.append(word_tokenize(row))
aftercleaning = [x for x in sentence if x != []]
iterations = list(itertools.chain.from_iterable(aftercleaning))

trigrams = nltk.trigrams(iterations)
f_dist = nltk.FreqDist(trigrams)
f_dist

FreqDist({('everi', 'movi', 'come'): 1,
          ('movi', 'come', 'truli'): 1,
          ('come', 'truli', 'make'): 1,
          ('truli', 'make', 'impact'): 1,
          ('make', 'impact', 'joaquin'): 1,
          ('impact', 'joaquin', 'perform'): 1,
          ('joaquin', 'perform', 'scenographi'): 1,
          ('perform', 'scenographi', 'brillianc'): 1,
          ('scenographi', 'brillianc', 'grotesqu'): 1,
          ('brillianc', 'grotesqu', 'hunt'): 1,
          ('grotesqu', 'hunt', 'cring'): 1,
          ('hunt', 'cring', 'hard'): 1,
          ('cring', 'hard', 'watch'): 1,
          ('hard', 'watch', 'time'): 1,
          ('watch', 'time', 'mesmer'): 1,
          ('time', 'mesmer', 'wont'): 1,
          ('mesmer', 'wont', 'blink'): 1,
          ('wont', 'blink', 'eye'): 1,
          ('blink', 'eye', 'watch'): 1,
          ('eye', 'watch', 'tragic'): 1,
          ('watch', 'tragic', 'serious'): 1,
          ('tragic', 'serious', 'funni'): 1,
          ('serious', 'funni', 'moment

**1.2 Probability of Bi grams**

In [9]:
from collections import Counter
grams = nltk.bigrams(iterations)
f_dist = nltk.FreqDist(grams)
bi_dict = dict(f_dist)
for word in bi_dict:
  print( str(word) + ':' + str(bi_dict[word] / iterations.count(word[0])))

Streaming output truncated to the last 5000 lines.
('inspir', 'three'):0.09090909090909091
('three', 'peopl'):1.0
('peopl', 'gun'):0.013888888888888888
('gun', 'duel'):0.14285714285714285
('duel', 'gold'):1.0
('gold', 'rate'):1.0
('rate', 'score'):0.07142857142857142
('score', '1010'):0.041666666666666664
('1010', 'grade'):0.14285714285714285
('grade', 'realli'):0.3333333333333333
('realli', 'dont'):0.024390243902439025
('understand', 'mani'):0.05555555555555555
('mani', 'peopl'):0.027777777777777776
('consid', 'masterpiec'):0.1
('masterpiec', 'simpli'):0.03125
('simpli', 'hate'):0.2
('hate', 'movi'):0.1
('movi', 'sinc'):0.0031847133757961785
('sinc', 'first'):0.25
('first', 'minut'):0.05263157894736842
('minut', 'mayb'):0.14285714285714285
('mayb', 'sensibl'):0.09090909090909091
('sensibl', 'feel'):1.0
('feel', 'empti'):0.034482758620689655
('empti', 'joke'):0.2857142857142857
('charact', 'annoy'):0.012048192771084338
('annoy', 'screenplay'):0.25
('screenplay', 'annoy'):0.111111111111

**1.3 Noun Phrases and relative probabilities of each review in tems of other reviews**

In [10]:
data_set = ''
n= []
i = 1
for x in data_frame['After Lemmatization']:
  data_set = data_set + x
  value = 'Review-' + str(i)
  n.append(value)
  i += 1

In [11]:
from textblob import TextBlob
noun = []
freq = []
for row in data_frame['After Lemmatization']:
  b = TextBlob(row)
  for nouns_words in b.noun_phrases:
    noun.append(nouns_words)
for x in noun:
  nounphrase_freq = []
  for row in data_frame['After Lemmatization']:
    nounphrase_freq.append(row.count(x) / data_set.count(x))
  freq.append(nounphrase_freq)
noun_phrases_data_frame = pd.DataFrame(freq).T
noun_phrases_data_frame.columns = list(noun)
noun_phrases_data_frame.n = n
noun_phrases_data_frame

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  from ipykernel import kernelapp as app


,everi movi,impact joaquin,scenographi brillianc grotesqu hunt,time mesmer wont blink eye,serious funni moment emot rollercoast sometim multipl emot,typic actionriddl,superhero movi,proper psycholog thrillerdrama singl,alon isol truli relat,sorri charact lot peopl,encourag violenc truli movi encourag everi,u becom,everyon respect,truli masterpiec,hollywood film,film decad truli,comic book willingli realist,direct cinematographi music act peopl surpris,violent necess messag societi reflect underappreciatedunrecognizedbulli peopl,someth way show class differ,corrupt rich talent rule,differ believ,multipl joke,bitter way film show,peopl comfort peopl consid wake,perfect film,joaquin phoenix,fearless stun emot depth physic imposs,referenc heath ledger oscarwin,dark knight,wide consid definit liveact portray joke,fact everyon,stun phoenix,imposs portray,potenti exce,dark knight clown princ crime,end fall,short way,short joke,time happi hope plea ignor complaint pernici violenc embarrass,...,movi good dont,wrong cinematographi amaz act,brilliant stori bit,slow pace annoy time,deaf flaw way,compar batman,dark knight movi,compens fanci wa,caus joke,fanci cri,liter creat limb account,time order attempt,peopl terribl film noth good,poorli creat stori awkwardli,grace joaquin phoenix act aw charact aw stori couldn,movi end,everi twist film end credit,final free date,great movi,describ distress pain sorrow depress twist oddli beauti joaquin,act anoth level,everi scene everyth,liter blew offphoenix,someth cinema,long time movi,imagin theatr,full peopl india clap film,deserv everi applaus,movi wont,charact need,mental physic suffer life time appreci rate believ lot pp experienc circumst henc sympathi,surpris idea play joke ballet toilet hilari,disturb littl,extrem uncomfort mind wander,young white disturb men senselessli murder peopl countri dont,illicit behavior stabl individu,emot disturb peopl parent plea awar content endors film,joke movi,artist gritti,brilliant oscar
0,1.0,1.0,1.0,1.0,1.0,1.0,0.2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,1.0,1.0,0.111111,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.0,0.

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

**2.1- TF-IDF**

In [12]:
import math
def tfidf_value(row,word):
  n = len(row.split(" "))
  tf = row.count(word)/n
  idf = 0
  if(tf!=0):
    idf = math.log(n)/row.count(word)
  else:
    return 0;
  return tf*idf

rows = data_frame["Spelling Correction"].values.tolist()
tokens = set([j for i in rows for j in i.split(" ")])
tfidf = pd.DataFrame(tokens,columns=["token"])
count = 0
for i in rows:
  tfidf[str(count)] = tfidf["token"].apply(lambda x : tfidf_value(i,x))
  count+=1

tfidf

,token,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
0,,0.081918,0.093937,0.055855,0.08065,0.083231,0.058163,0.110774,0.035297,0.070931,0.0,0.0,0.061364,0.244136,0.188504,0.074911,0.244136,0.040702,0.071881,0.0,0.058773,0.074911,0.064221,0.020991,0.128755,0.0,0.069111,0.039377,0.0,0.072861,0.0,0.054775,0.093937,0.08065,0.116114,0.093937,0.0,0.108304,0.04126,0.0,...,0.16666,0.06348,0.132419,0.095726,0.197304,0.052748,0.03677,0.160576,0.0,0.15497,0.116114,0.0,0.019607,0.277987,0.028467,0.070008,0.040161,0.057567,0.058773,0.25993,0.0,0.048333,0.16666,0.103716,0.110774,0.173287,0.149787,0.055309,0.072861,0.125311,0.092222,0.298627,0.07824,0.099542,0.128755,0.128755,0.25993,0.088992,0.21799,0.230259
1,curtains,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.070931,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
2,million,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.00000,0.00000,0.000000,0.095726,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
3,wants,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.058773,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
4,master,0.000000,0.000000,0.055855,0.00000,0.000000,0.000000,0.110774,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.040702,0.071881,0.0,0.000000,0.000000,0.064221,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.093937,0.0,0.000000,0.00000,0.0,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.03677,0.000000,0.0,0.00000,0.000000,0.0,0.019607,0.000000,0.028467,0.070008,0.000000,0.000000,0.000000,0.00000,0.0,0.048333,0.00000,0.103716,0.000000,0.000000,0.000000,0.000000,0.072861,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2828,types,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000

**2.2 Ranking using Cosine Similarity**

In [50]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [51]:
query = 'The perfect portrait of a decline to insanity" & Joaquin Phoenix portrays this flawlessly in a performance of a lifetime. Choreographed impeccably by Todd Phillips & is nothing short of breathtaking. The cinematography & backdrop ideology of such an original film for such a complex character is only complimented by the superb soundtrack & acting. A hauntingly beautiful film & A Definate 10/10. Edit: Bring on part 2 plssssss!. We all want it!.'
X = word_tokenize(query)
s = stopwords.words('english') 
set_X = {w for w in X if not w in s}

In [52]:
def vector(set_X, set_Y):
  r_vec = set_X.union(set_Y)
  list_1 =[]; list_2 =[]
  for w in r_vec: 
      if w in set_X: list_1.append(1)
      else: list_1.append(0) 
      if w in set_Y: list_2.append(1) 
      else: list_2.append(0)
  return r_vec, list_1, list_2

In [54]:
def cos(r_vec, list_1, list_2):
  cc = 0
  for i in range(len(r_vec)): 
        cc+= list_1[i]*list_2[i] 
  cos = cc / float((sum(list_1)*sum(list_2))**0.5) 
  return cos

In [55]:
sim_list = []
for row in data_frame['Spelling Correction']:
  list_Y = word_tokenize(row)
  set_Y = {w for w in list_Y if not w in s}
  r_vec, list_1, list_2 = vector(set_X, set_Y)
  try:
    sim = cos(r_vec, list_1, list_2)
  except ZeroDivisionError:
    sim = 'None'
  sim_list.append(sim)
cosine_data_frame = pd.DataFrame(list(zip(data_frame['Review'],sim_list)), columns=['Reviews','Cosine Similarity'])
cosine_data_frame

,Reviews,Cosine Similarity
0,"Every once in a while a movie comes, that trul...",0.0434783
1,This is a movie that only those who have felt ...,0.0256664
2,"Truly a masterpiece, The Best Hollywood film o...",0.0761387
3,Joaquin Phoenix gives a tour de force performa...,0.0227508
4,Most of the time movies are anticipated like t...,0.0230266
...,...,...
195,This movie won't be felt only by those who suf...,0.0294884
196,No surprises here. This was just the same idea...,0
197,Disturbing with very little entertainment valu...,0.0245737
198,Just went and watched the joker movie and it h...,0


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:
# The GitHub link of your final csv file

# Link: 
https://github.com/Akshitasama/akshita_INFO5731_Spring2021/blob/fcd0b2bca8dfdbbff696b1c1d03548e52ab2da83/Reviews.csv